# Tesla Supercharger Address Scraping & Geocoding
End-to-end Jupyter Notebook documenting HTML scraping, address normalization, and Google Maps geocoding.

## 1. Environment Setup

In [ ]:

from bs4 import BeautifulSoup
import pandas as pd
import requests
import os


## 2. Load Local Tesla Supercharger HTML File

In [ ]:

file_path = "/Users/judycheng/Downloads/tesla supercharger.html"

with open(file_path, "r", encoding="utf-8") as f:
    html_content = f.read()


## 3. Parse HTML

In [ ]:

soup = BeautifulSoup(html_content, "html.parser")


## 4. Extract Address Elements

In [ ]:

divs = soup.find_all("div", class_="subregion_location_addressLine1")
addresses = [div.get_text(strip=True) for div in divs]


## 5. Pair Address Lines

In [ ]:

if len(addresses) % 2 != 0:
    addresses.append("")

paired_addresses = [
    (addresses[i], addresses[i + 1])
    for i in range(0, len(addresses), 2)
]


## 6. Create Address DataFrame

In [ ]:

df = pd.DataFrame(paired_addresses, columns=["Address Part 1", "Address Part 2"])
df.to_csv("paired_addresses.csv", index=False, encoding="utf-8")
df.head()


## 7. Construct Full Address Field

In [ ]:

df["Full Address"] = df["Address Part 1"] + ", " + df["Address Part 2"]
df.to_csv("full_addresses.csv", index=False, encoding="utf-8")
df[["Full Address"]].head()


## 8. Google Geocoding (Full Address)

In [ ]:

API_KEY = "REPLACE_WITH_YOUR_API_KEY"

df = pd.read_csv("/Users/judycheng/Downloads/full_addresses.csv")

def get_coordinates(address):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": address, "key": API_KEY}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        if data["status"] == "OK":
            loc = data["results"][0]["geometry"]["location"]
            return loc["lat"], loc["lng"]
    return None, None

df["Latitude"], df["Longitude"] = zip(*df["Full Address"].apply(get_coordinates))
df.to_csv("addresses_with_coords.csv", index=False)
df.head()


## 9. ZIP-Code Geocoding for EV Population Dataset

In [ ]:

import csv

API_KEY = "REPLACE_WITH_YOUR_API_KEY"

def get_zip_coordinates(postal_code):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={postal_code}&key={API_KEY}"
    response = requests.get(url)
    data = response.json()

    if data["status"] == "OK" and data["results"]:
        loc = data["results"][0]["geometry"]["location"]
        return loc["lat"], loc["lng"]
    return None, None

input_csv = "/Users/judycheng/Downloads/Electric_Vehicle_Population_Data 16.csv"
output_csv = "/Users/judycheng/Desktop/coordinates_output.csv"

with open(input_csv, newline="", encoding="utf-8") as infile,      open(output_csv, "w", newline="", encoding="utf-8") as outfile:

    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames + ["Latitude", "Longitude"]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        lat, lng = get_zip_coordinates(row["Postal Code"])
        row["Latitude"] = lat
        row["Longitude"] = lng
        writer.writerow(row)

print(f"Results saved to {output_csv}")


## 10. Outputs
- paired_addresses.csv
- full_addresses.csv
- addresses_with_coords.csv
- coordinates_output.csv

These files support geospatial analysis for Supercharger coverage modeling.